<a href="https://colab.research.google.com/github/saks0106/Frequent-Lookouts/blob/main/Column_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [7]:
X_train

,age,gender,fever,cough,city
52,47,Female,100.0,Strong,Bangalore
33,26,Female,98.0,Mild,Kolkata
83,17,Female,104.0,Mild,Kolkata
11,65,Female,98.0,Mild,Mumbai
92,82,Female,102.0,Strong,Kolkata
...,...,...,...,...,...
54,60,Female,99.0,Mild,Mumbai
3,31,Female,98.0,Mild,Kolkata
56,71,Male,NaN,Strong,Kolkata
30,15,Male,101.0,Mild,Delhi


## 1. Traditional Method - Different Techniques from different feature(s) values

In [8]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [9]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [10]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [11]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [ ]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

## Using ColumnTransformer

In [12]:
from sklearn.compose import ColumnTransformer

In [13]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough') # remainder - what to do with remaining columns

In [17]:
transformer.fit_transform(X_train).shape

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [21]:
transformer.transform(X_test).shape

(20, 7)

In [26]:
X_train_transformed = transformer.fit_transform(X_train)
pd.DataFrame(X_train_transformed)


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6
0,100.000000,1.0,0.0,0.0,0.0,0.0,47.0
1,98.000000,0.0,0.0,0.0,1.0,0.0,26.0
2,104.000000,0.0,0.0,0.0,1.0,0.0,17.0
3,98.000000,0.0,0.0,0.0,0.0,1.0,65.0
4,102.000000,1.0,0.0,0.0,1.0,0.0,82.0
...,...,...,...,...,...,...,...
75,99.000000,0.0,0.0,0.0,0.0,1.0,60.0
76,98.000000,0.0,0.0,0.0,1.0,0.0,31.0
77,100.708333,1.0,1.0,0.0,1.0,0.0,71.0
78,101.000000,0.0,1.0,1.0,0.0,0.0,15.0


In [ ]:
X_test_transformed = transformer.transform(X_test)

In [23]:
X_train

,age,gender,fever,cough,city
52,47,Female,100.0,Strong,Bangalore
33,26,Female,98.0,Mild,Kolkata
83,17,Female,104.0,Mild,Kolkata
11,65,Female,98.0,Mild,Mumbai
92,82,Female,102.0,Strong,Kolkata
...,...,...,...,...,...
54,60,Female,99.0,Mild,Mumbai
3,31,Female,98.0,Mild,Kolkata
56,71,Male,NaN,Strong,Kolkata
30,15,Male,101.0,Mild,Delhi
